In [1]:
import pandas as pd
from getbibleverse import verse_lookup

In [18]:
def store_verses(*passage):
    
    # create empty df to store verses
    verses_df = pd.DataFrame(columns=["bookname", "chapter", "verse", "text"])
    df = pd.DataFrame([])
    
    # create empty list to store input separately
    books = []
    chapters = []
    verses = []
    
    # break down input into separate lists
    for entry in passage:
        book, ch_ver = entry.split(" ")
        chapter, verse_range = ch_ver.split(":")
        
        books.append(book)
        chapters.append(chapter)
        verses.append(verse_range)
        
    for i in range(len(books)):
        data = pd.json_normalize(verse_lookup(books[i], chapters[i], verses[i]))
        verses_df = pd.concat([verses_df, data], ignore_index=True)
        
    df['book'] = verses_df["bookname"].unique()
    df['chapter'] = verses_df['chapter'].unique()
    df['verses'] = [i for i in verses]
    df['location'] = df["book"] + " " + df["chapter"] + ":" + df["verses"]
    
    # how do I get text
    

    return df

In [19]:
store_verses("Genesis 1:1-10", "Exodus 3:5-15")

,book,chapter,verses,location
0,Genesis,1,1-10,Genesis 1:1-10
1,Exodus,3,5-15,Exodus 3:5-15
